In [1]:
#импортируем библиотеки
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#Загружаем данные
train = pd.read_csv('shift_ml_2025_train.csv')
target = train['итоговый_статус_займа']
test = pd.read_csv('shift_ml_2025_test.csv')

train.describe()
train.head(5)

#Отбираем топ признаки при помощи RandomForestClassifier
X = train.drop(columns=["id", "итоговый_статус_займа"], errors='ignore') 
y = train["итоговый_статус_займа"]

X_num = X.select_dtypes(include=["number"]).fillna(0)
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X_num, y, test_size=0.3, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train_rf, y_train_rf)
importances = pd.Series(model.feature_importances_, index=X_num.columns)
importances.sort_values(ascending=False)
top_features = importances.sort_values(ascending=False).head(43).index

#Оставим признаки, которые описывают хотя бы 1%
top_features = importances.sort_values(ascending=False).head(43).index
X = pd.concat([train[top_features], train['итоговый_статус_займа']], axis=1)

# EDA / трансформации

def safe_drop(df, cols):
    return df.drop(columns=[c for c in (cols if isinstance(cols, (list,tuple)) else [cols]) if c in df.columns], errors='ignore')

# удаляем коррелированные колонки
X = safe_drop(X, ['коэфф_акционных_зачислений_в_остатке','коэфф_исходного_платежа','общий_лимит_по_возоб_счету'])

# процент_счетов_без_просрочек заполним 100
if 'процент_счетов_без_просрочек' in X.columns:
    X['процент_счетов_без_просрочек'] = X['процент_счетов_без_просрочек'].fillna(100)

# Кол-во счетов без нарушений заполним пропуски 0
if 'кол-во_счетов_без_нарушений' in X.columns:
    X['кол-во_счетов_без_нарушений'] = X['кол-во_счетов_без_нарушений'].fillna(0)

# Логарифмируем пдн из-за скошенности
if 'пдн' in X.columns:
    X['пдн'] = np.log1p(abs(X['пдн']))

# Прологарифмрованный Коэфф_загрузки_возобновляемого_счета из-за скошенности
if 'коэфф_загрузки_возобновляемого_счета' in X.columns:
    X['коэфф_загрузки_возобновляемого_счета'] = np.log1p(X['коэфф_загрузки_возобновляемого_счета'].fillna(0))

# Удаляем указанные колонки, обработаем их позже
X = safe_drop(X, ['кол-во_открытых_счетов','аннуитет'])
if 'верхний_порог_рейтинга_заемщика' in X.columns and 'нижний_порог_рейтинга_заемщика' in X.columns:
    X['средний_порог_рейтинга_заемщика'] = (X['верхний_порог_рейтинга_заемщика'] + X['нижний_порог_рейтинга_заемщика'])//2
X = safe_drop(X, ['верхний_порог_рейтинга_заемщика','нижний_порог_рейтинга_заемщика'])
X = safe_drop(X, ['кредитный_баланс_без_ипотеки'])

# кредитный лимит и годовой доход 
if 'кредитный_лимит' in X.columns:
    X['кредитный_лимит'] = X['кредитный_лимит'].fillna(X['кредитный_лимит'].median())
if 'годовой_доход' in X.columns:
    X['годовой_доход'] = np.log1p(X['годовой_доход'].fillna(X['годовой_доход'].median()))

# Заполним пропуски
if 'лимит_по_картам' in X.columns:
    X['лимит_по_картам'] = X['лимит_по_картам'].fillna(X['лимит_по_картам'].median())

# удаляем перечисленные колонки из-за коррелированности, сложности интерпритации
X = safe_drop(X, ['кол-во_месяцев_с_первого_возобновляемого_счета',
                  'суммарная_доступная_сумма_займа_по_картам',
                  'средний_баланс_текущих_счетов',
                  'кол-во_месяцев_с_первого_аннуитетного_счета',
                  'соотношение_баланса_к_лимиту_по_картам',
                  'лимит_по_картам','оборотный_баланс','общая_сумма_на_счетах'])

# логарифмируем сумму займа
if 'сумма_займа' in X.columns:
    X['сумма_займа'] = np.log1p(X['сумма_займа'])

# кол-во счетов лог
if 'кол-во_счетов' in X.columns:
    X['кол-во_счетов'] = np.log1p(X['кол-во_счетов'])

X = safe_drop(X, ['кол-во_возобновляемых_счетов','кол-во_аннуитетных_счетов','кол-во_месяцев_с_последнего_карты'])

# кол-во_мес_с_последней_заявки заполним 0 и лог
if 'кол-во_мес_с_последней_заявки' in X.columns:
    X['кол-во_мес_с_последней_заявки'] = X['кол-во_мес_с_последней_заявки'].fillna(0)
    X['кол-во_мес_с_последней_заявки'] = np.log1p(X['кол-во_мес_с_последней_заявки'])

# месяцы с последнего счета лог
if 'кол-во_месяцев_с_последнего_счета' in X.columns:
    X['кол-во_месяцев_с_последнего_счета'] = np.log1p(X['кол-во_месяцев_с_последнего_счета'].fillna(0))

# Кол-во_без_нарушений/Кол-во_счетов 
if 'кол-во_счетов_без_нарушений' in X.columns and 'кол-во_счетов' in X.columns:
    X['Кол-во_без_нарушений/Кол-во_счетов'] = np.log1p(X['кол-во_счетов_без_нарушений'])/X['кол-во_счетов']
X = safe_drop(X, ['кол-во_открытых_возобновляемых_счетов','кол-во_счетов_без_нарушений'])

# процент_счетов_без_просрочек лог и удаление (как у тебя)
if 'процент_счетов_без_просрочек' in X.columns:
    X['процент_счетов_без_просрочек'] = np.log1p(X['процент_счетов_без_просрочек'])
X = safe_drop(X, ['процент_счетов_без_просрочек','кол-во_возобновляемых_счетов_с_балансом_более_0',
                  'кол-во_карт_без_нарушений','кол-во_активных_возобновляемых_счетов',
                  'процент_счетов_прев_75_лимита'])

# заполняем пдн и другие
if 'пдн' in X.columns:
    X['пдн'] = X['пдн'].fillna(0)
X = safe_drop(X, ['кол-во_карт'])
if 'кол-во_месяцев_с_последнего_счета' in X.columns:
    X['кол-во_месяцев_с_последнего_счета'] = X['кол-во_месяцев_с_последнего_счета'].fillna(0)
if 'лимит_по_аннуитетным_счетам' in X.columns:
    X['лимит_по_аннуитетным_счетам'] = X['лимит_по_аннуитетным_счетам'].fillna(0)
    X['лимит_по_аннуитетным_счетам'] = np.log1p(X['лимит_по_аннуитетным_счетам'])
if 'индекс_проживания' in X.columns:
    X['индекс_проживания'] = X['индекс_проживания'].fillna(X['индекс_проживания'].median())
if 'Кол-во_без_нарушений/Кол-во_счетов' in X.columns:
    X['Кол-во_без_нарушений/Кол-во_счетов'] = X['Кол-во_без_нарушений/Кол-во_счетов'].fillna(0)


if 'кол-во_месяцев_с_последней_просрочки' in X.columns:
    X['кол-во_месяцев_с_последней_просрочки'] = X['кол-во_месяцев_с_последней_просрочки'].fillna(0)

# суммарное_кол-во_счетов - Сделаем одно общее поле для большинства счетов
from sklearn.preprocessing import MinMaxScaler
cols_to_sum = [
    'кол-во_возоб_счетов_за_2_года',
    'кол-во_возоб_счетов_за_год',
    'кол-во_возобновляемых_счетов_с_балансом_более_0',
    'кол-во_активных_возобновляемых_счетов',
    'кол-во_открытых_счетов_за_2_года',
    'кол-во_открытых_счетов_за_полгода',
    'кол-во_счетов_за_посл_год'
]
cols_to_sum_present = [c for c in cols_to_sum if c in train.columns]
if cols_to_sum_present:
    scaler = MinMaxScaler()
    _ = scaler.fit_transform(train[cols_to_sum_present])
    train['суммарное_кол-во_счетов'] = train[cols_to_sum_present].sum(axis=1)

X = pd.concat([
    X,
    train['суммарное_кол-во_счетов'] if 'суммарное_кол-во_счетов' in train.columns else pd.Series(0, index=X.index),
    train['кол-во_заявок_за_полгода'].fillna(0) if 'кол-во_заявок_за_полгода' in train.columns else pd.Series(0, index=X.index),
    train['кол-во_месяцев_с_первого_возобновляемого_счета'].fillna(0) if 'кол-во_месяцев_с_первого_возобновляемого_счета' in train.columns else pd.Series(0, index=X.index),
    train['процент_счетов_прев_75_лимита'].fillna(0) if 'процент_счетов_прев_75_лимита' in train.columns else pd.Series(0, index=X.index)
], axis=1)

# процентная ставка fillna 
if 'процентная_ставка' in X.columns:
    X['процентная_ставка'] = X['процентная_ставка'].fillna(X['процентная_ставка'].median())

# добавляем категориальные 
categorical_cols = train.select_dtypes(include=['object']).columns.tolist()
X = pd.concat([X, train[categorical_cols]], axis=1)

missing = X.isnull().sum().sort_values()
missing

#  преобразования даты в декады
# срок_займа: извлекаем первую цифру
if 'срок_займа' in X.columns:
    X['срок_займа'] = X['срок_займа'].str.split().str[0].astype(int)

# рейтинг и допрейтинг — OrdinalEncoder 
rating_order = ['А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж']
dop_rating_order = [f"{letter}{num}" for letter in rating_order for num in range(1, 6)]
# создаём отдельный энкодер enc_rating чтобы не перезаписывать переменные
enc_rating = OrdinalEncoder(categories=[rating_order, dop_rating_order])
if all(c in X.columns for c in ['рейтинг','допрейтинг']):
    X[['рейтинг', 'допрейтинг']] = enc_rating.fit_transform(X[['рейтинг','допрейтинг']])

# стаж — OrdinalEncoder 
if 'стаж' in X.columns:
    X['стаж'] = X['стаж'].fillna('не указан')
    experience_order = [['не указан','< 1 года', '1 год', '2 года', '3 года', '4 года', '5 лет', '6 лет', '7 лет', '8 лет', '9 лет', '10+ лет']]
    enc1 = OrdinalEncoder(categories=experience_order)
    X['стаж'] = enc1.fit_transform(X[['стаж']])

# платежный график - пустой столбец с n
X = safe_drop(X, 'платежный_график')

# дата_первого_займа -> год, декада -> OrdinalEncoder 
if 'дата_первого_займа' in X.columns:
    X['дата_первого_займа'] = pd.to_datetime(X['дата_первого_займа'])
    X['год'] = X['дата_первого_займа'].dt.year
    X['декада'] = (X['год'] // 10) * 10
    X = X.dropna(subset=['декада'])
    enc_dek = OrdinalEncoder(categories=[[1930,1940,1950,1960,1970,1980,1990,2000,2010,2020]])
    X['декада'] = enc_dek.fit_transform(X[['декада']])
    X = safe_drop(X, 'дата_первого_займа')

# первоначальный_статус_займа - OrdinalEncoder
if 'первоначальный_статус_займа' in X.columns:
    enc_init = OrdinalEncoder(categories=[['а','б']])
    X['первоначальный_статус_займа'] = enc_init.fit_transform(X[['первоначальный_статус_займа']])

# удаляем пени_за_дефолт (утечка)
X = safe_drop(X, 'пени_за_дефолт')

# совокупный_статус_подтверждения_доходов_заемщиков -> fillna
if 'совокупный_статус_подтверждения_доходов_заемщиков' in X.columns:
    X['совокупный_статус_подтверждения_доходов_заемщиков'] = X['совокупный_статус_подтверждения_доходов_заемщиков'].fillna('Не подтвержден')
    X['совокупный_статус_подтверждения_доходов_заемщиков'] = X['совокупный_статус_подтверждения_доходов_заемщиков'].astype(str)

X = safe_drop(X, 'особая_ситуация')

if 'процентная_ставка' in X.columns:
    X = X.dropna(subset=['процентная_ставка'])
if 'кол-во_счетов' in X.columns:
    X = X.dropna(subset=['кол-во_счетов'])

# бинарный индикатор просрочки и лог суммы просрочек
if 'сумма_выплат_по_просрочкам' in X.columns:
    X['бинарные_идентификатор_суммы_просрочки'] = (X['сумма_выплат_по_просрочкам'] > 0).astype(int)
    X['сумма_выплат_по_просрочкам'] = np.log1p(X['сумма_выплат_по_просрочкам'].fillna(0))

# отделяем y и удаляем таргет столбец
if 'итоговый_статус_займа' in X.columns:
    y = X['итоговый_статус_займа']
    X = X.drop('итоговый_статус_займа', axis=1)

# OneHot 
# Владение жильем
encoder_vl = OneHotEncoder(sparse_output=False, dtype=int, handle_unknown='ignore')
if 'владение_жильем' in X.columns:
    encoded_array = encoder_vl.fit_transform(X[['владение_жильем']])
    encoded_cols = encoder_vl.get_feature_names_out(['владение_жильем'])
    train_encoded = pd.DataFrame(encoded_array, columns=encoded_cols, index=X.index)
    X = pd.concat([X, train_encoded], axis=1)
    X = X.drop('владение_жильем', axis=1)

# Цель займа 
encoder_goal = OneHotEncoder(sparse_output=False, dtype=int, handle_unknown='ignore')
if 'цель_займа' in X.columns:
    encoded_array = encoder_goal.fit_transform(X[['цель_займа']])
    encoded_cols = encoder_goal.get_feature_names_out(['цель_займа'])
    train_encoded1 = pd.DataFrame(encoded_array, columns=encoded_cols, index=X.index)
    X = pd.concat([X, train_encoded1], axis=1)
    X = X.drop('цель_займа', axis=1)

# Подтвержден ли доход
encoder5 = OneHotEncoder(sparse_output=False, dtype=int, handle_unknown='ignore')
if 'подтвержден_ли_доход' in X.columns:
    encoded_array5 = encoder5.fit_transform(X[['подтвержден_ли_доход']])
    encoded_cols5 = encoder5.get_feature_names_out(['подтвержден_ли_доход'])
    train_encoded5 = pd.DataFrame(encoded_array5, columns=encoded_cols5, index=X.index)
    X = pd.concat([X, train_encoded5], axis=1)
    X = X.drop('подтвержден_ли_доход', axis=1)

# Совокупный статус подтверждения доходов
encoder3 = OneHotEncoder(sparse_output=False, dtype=int, handle_unknown='ignore')
if 'совокупный_статус_подтверждения_доходов_заемщиков' in X.columns:
    encoded_array3 = encoder3.fit_transform(X[['совокупный_статус_подтверждения_доходов_заемщиков']])
    encoded_cols3 = encoder3.get_feature_names_out(['совокупный_статус_подтверждения_доходов_заемщиков'])
    train_encoded3 = pd.DataFrame(encoded_array3, columns=encoded_cols3, index=X.index)
    X = pd.concat([X, train_encoded3], axis=1)
    X = X.drop('совокупный_статус_подтверждения_доходов_заемщиков', axis=1)

# Тип предоставления кредита
encoder4 = OneHotEncoder(sparse_output=False, dtype=int, handle_unknown='ignore')
if 'тип_предоставления_кредита' in X.columns:
    encoded_array4 = encoder4.fit_transform(X[['тип_предоставления_кредита']])
    encoded_cols4 = encoder4.get_feature_names_out(['тип_предоставления_кредита'])
    train_encoded4 = pd.DataFrame(encoded_array4, columns=encoded_cols4, index=X.index)
    X = pd.concat([X, train_encoded4], axis=1)
    X = X.drop('тип_предоставления_кредита', axis=1)

# пос_стоп_фактор
encoder6 = OneHotEncoder(sparse_output=False, dtype=int, handle_unknown='ignore')
if 'пос_стоп_фактор' in X.columns:
    encoded_array6 = encoder6.fit_transform(X[['пос_стоп_фактор']])
    encoded_cols6 = encoder6.get_feature_names_out(['пос_стоп_фактор'])
    train_encoded6 = pd.DataFrame(encoded_array6, columns=encoded_cols6, index=X.index)
    X = pd.concat([X, train_encoded6], axis=1)
    X = X.drop('пос_стоп_фактор', axis=1)

# юридический_статус
encoder7 = OneHotEncoder(sparse_output=False, dtype=int, handle_unknown='ignore')
if 'юридический_статус' in X.columns:
    encoded_array7 = encoder7.fit_transform(X[['юридический_статус']])
    encoded_cols7 = encoder7.get_feature_names_out(['юридический_статус'])
    train_encoded7 = pd.DataFrame(encoded_array7, columns=encoded_cols7, index=X.index)
    X = pd.concat([X, train_encoded7], axis=1)
    X = X.drop('юридический_статус', axis=1)

# тип_займа
encoder2 = OneHotEncoder(sparse_output=False, dtype=int, handle_unknown='ignore')
if 'тип_займа' in X.columns:
    encoded_array2 = encoder2.fit_transform(X[['тип_займа']])
    encoded_cols2 = encoder2.get_feature_names_out(['тип_займа'])
    train_encoded2 = pd.DataFrame(encoded_array2, columns=encoded_cols2, index=X.index)
    X = pd.concat([X, train_encoded2], axis=1)
    X = X.drop('тип_займа', axis=1)

# новые колонки - cмотрим комбинационные зависимости
new_cols = {}
if 'кол-во_ипотек' in train.columns:
    new_cols['log_ипотек'] = np.log1p(train['кол-во_ипотек'])
if 'кредитный_баланс_без_ипотеки' in train.columns:
    new_cols['log_кредит_без_ипотеки'] = np.log1p(train['кредитный_баланс_без_ипотеки'])
if 'аннуитет' in train.columns:
    new_cols['log_аннуитет'] = np.log1p(train['аннуитет'])
if all(c in X.columns for c in ['процентная_ставка','срок_займа','сумма_займа']):
    new_cols['процент_ставка_срок_сумма'] = X['процентная_ставка'] * X['срок_займа'] * X['сумма_займа']
if 'грейд_на_детерминаторе' in train.columns:
    new_cols['грейд_на_детерминаторе'] = train['грейд_на_детерминаторе']
if all(c in X.columns for c in ['годовой_доход','сумма_займа']):
    new_cols['доход_к_займу'] = X['годовой_доход'] / X['сумма_займа'].replace(0, np.nan)

if new_cols:
    X = pd.concat([X, pd.DataFrame(new_cols, index=X.index)], axis=1)
if 'процентная_ставка' in X.columns:
    X = X.dropna(subset=['процентная_ставка'])

X.columns = X.columns.astype(str)
missing=X.isnull().sum()
missing
X['log_ипотек']=X['log_ипотек'].fillna(X['log_ипотек'].median())
X['log_кредит_без_ипотеки']=X['log_кредит_без_ипотеки'].fillna(X['log_кредит_без_ипотеки'].median())
X['кол-во_месяцев_с_последней_карты']=X['кол-во_месяцев_с_последней_карты'].fillna(X['кол-во_месяцев_с_последней_карты'].median())
X['кол-во_месяцев_с_последнего_возобновляемого_счета']=X['кол-во_месяцев_с_последнего_возобновляемого_счета'].fillna(X['кол-во_месяцев_с_последнего_возобновляемого_счета'].median())
X['кол-во_открытых_счетов_за_2_года']=X['кол-во_открытых_счетов_за_2_года'].fillna(X['кол-во_открытых_счетов_за_2_года'].median())
# ====== Target encoding региона и профессии как у тебя (mapping из X_R -> X_test) ======
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_R = pd.concat([X_train_full, y_train_full], axis=1)

#вычисляем mean по региону на X_R и мапим на X_train_full и X_test_full
global_mean = X_R['итоговый_статус_займа'].mean()
region_means = X_R.groupby('регион')['итоговый_статус_займа'].mean() if 'регион' in X_R.columns else pd.Series()
prof_mean = X_R.groupby('профессия_заемщика')['итоговый_статус_займа'].mean() if 'профессия_заемщика' in X_R.columns else pd.Series()

if 'регион' in X_train_full.columns:
    X_train_full['регион_target_encoded'] = X_train_full['регион'].map(region_means)
    X_test_full['регион_target_encoded'] = X_test_full['регион'].map(region_means).fillna(global_mean)
    region_counts = X_R['регион'].value_counts()
    X_train_full['регион_freq_encoded'] = X_train_full['регион'].map(region_counts)
    X_test_full['регион_freq_encoded'] = X_test_full['регион'].map(region_counts).fillna(0)
    X_train_full = X_train_full.drop('регион', axis=1)
    X_test_full = X_test_full.drop('регион', axis=1)
if 'профессия_заемщика' in X_train_full.columns:
    X_train_full = X_train_full.drop('профессия_заемщика', axis=1)
    X_test_full = X_test_full.drop('профессия_заемщика', axis=1)
#preprocessor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler

numeric_cols = X_train_full.select_dtypes(include=['int64','float64','int32']).columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', RobustScaler(), numeric_cols),
    ],
    remainder='passthrough'
)

# Применяем преобразования
X_train_scaled = preprocessor.fit_transform(X_train_full)
X_test_scaled = preprocessor.transform(X_test_full)




C:\Users\lego-\AppData\Local\Temp\ipykernel_19392\3076531231.py:18: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('shift_ml_2025_train.csv')
C:\Users\lego-\AppData\Local\Temp\ipykernel_19392\3076531231.py:201: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  X['дата_первого_займа'] = pd.to_datetime(X['дата_первого_займа'])


In [2]:
# выбираем CatBoostClassifer как модель с наибольшей описательной способностью по ROC AUC
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score

model = CatBoostClassifier(
    iterations=8000,
    learning_rate=0.03,
    depth=5,
    loss_function='Logloss',
    random_state=42,
    eval_metric='AUC',
    verbose=100,
    auto_class_weights='Balanced',
    task_type='GPU'
)
model.fit(
    X_train_scaled, y_train_full,
    eval_set=(X_test_scaled, y_test_full),
    use_best_model=True,
    early_stopping_rounds=100,
)

y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
roc_auc = roc_auc_score(y_test_full, y_pred_proba)
print(f"ROC AUC: {roc_auc:.4f}")

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7167753	best: 0.7167753 (0)	total: 167ms	remaining: 22m 15s
100:	test: 0.7422874	best: 0.7422874 (100)	total: 1.13s	remaining: 1m 28s
200:	test: 0.7462976	best: 0.7462976 (200)	total: 2.09s	remaining: 1m 21s
300:	test: 0.7483369	best: 0.7483369 (300)	total: 3.03s	remaining: 1m 17s
400:	test: 0.7498326	best: 0.7498326 (400)	total: 3.97s	remaining: 1m 15s
500:	test: 0.7509471	best: 0.7509471 (500)	total: 4.92s	remaining: 1m 13s
600:	test: 0.7518557	best: 0.7518557 (600)	total: 5.84s	remaining: 1m 11s
700:	test: 0.7525711	best: 0.7525711 (700)	total: 6.8s	remaining: 1m 10s
800:	test: 0.7531831	best: 0.7531831 (800)	total: 7.74s	remaining: 1m 9s
900:	test: 0.7536697	best: 0.7536697 (900)	total: 8.66s	remaining: 1m 8s
1000:	test: 0.7540881	best: 0.7540881 (1000)	total: 9.58s	remaining: 1m 6s
1100:	test: 0.7544754	best: 0.7544754 (1100)	total: 10.5s	remaining: 1m 5s
1200:	test: 0.7548474	best: 0.7548474 (1200)	total: 11.5s	remaining: 1m 4s
1300:	test: 0.7551461	best: 0.7551461 (13

In [3]:
#Преобразуем тестовые данные 
X = test[top_features]


X = safe_drop(X, ['коэфф_акционных_зачислений_в_остатке','коэфф_исходного_платежа','общий_лимит_по_возоб_счету'])


if 'процент_счетов_без_просрочек' in X.columns:
    X['процент_счетов_без_просрочек'] = X['процент_счетов_без_просрочек'].fillna(100)

# Кол-во счетов без нарушений заполним пропуски 0
if 'кол-во_счетов_без_нарушений' in X.columns:
    X['кол-во_счетов_без_нарушений'] = X['кол-во_счетов_без_нарушений'].fillna(0)

if 'пдн' in X.columns:
    X['пдн'] = np.log1p(abs(X['пдн']))

if 'коэфф_загрузки_возобновляемого_счета' in X.columns:
    X['коэфф_загрузки_возобновляемого_счета'] = np.log1p(X['коэфф_загрузки_возобновляемого_счета'].fillna(0))

X = safe_drop(X, ['кол-во_открытых_счетов','аннуитет'])
if 'верхний_порог_рейтинга_заемщика' in X.columns and 'нижний_порог_рейтинга_заемщика' in X.columns:
    X['средний_порог_рейтинга_заемщика'] = (X['верхний_порог_рейтинга_заемщика'] + X['нижний_порог_рейтинга_заемщика'])//2
X = safe_drop(X, ['верхний_порог_рейтинга_заемщика','нижний_порог_рейтинга_заемщика'])

if 'кредитный_лимит' in X.columns:
    X['кредитный_лимит'] = X['кредитный_лимит'].fillna(X['кредитный_лимит'].median())
if 'годовой_доход' in X.columns:
    X['годовой_доход'] = np.log1p(X['годовой_доход'].fillna(X['годовой_доход'].median()))

if 'лимит_по_картам' in X.columns:
    X['лимит_по_картам'] = X['лимит_по_картам'].fillna(X['лимит_по_картам'].median())

X = safe_drop(X, ['кол-во_месяцев_с_первого_возобновляемого_счета',
                  'суммарная_доступная_сумма_займа_по_картам',
                  'средний_баланс_текущих_счетов',
                  'кол-во_месяцев_с_первого_аннуитетного_счета',
                  'соотношение_баланса_к_лимиту_по_картам',
                  'лимит_по_картам','оборотный_баланс','общая_сумма_на_счетах'])

if 'сумма_займа' in X.columns:
    X['сумма_займа'] = np.log1p(X['сумма_займа'])

if 'кол-во_счетов' in X.columns:
    X['кол-во_счетов'] = np.log1p(X['кол-во_счетов'])

X = safe_drop(X, ['кол-во_возобновляемых_счетов','кол-во_аннуитетных_счетов','кол-во_месяцев_с_последнего_карты'])

if 'кол-во_мес_с_последней_заявки' in X.columns:
    X['кол-во_мес_с_последней_заявки'] = X['кол-во_мес_с_последней_заявки'].fillna(0)
    X['кол-во_мес_с_последней_заявки'] = np.log1p(X['кол-во_мес_с_последней_заявки'])

if 'кол-во_месяцев_с_последнего_счета' in X.columns:
    X['кол-во_месяцев_с_последнего_счета'] = np.log1p(X['кол-во_месяцев_с_последнего_счета'].fillna(0))

if 'кол-во_счетов_без_нарушений' in X.columns and 'кол-во_счетов' in X.columns:
    X['Кол-во_без_нарушений/Кол-во_счетов'] = np.log1p(X['кол-во_счетов_без_нарушений'])/X['кол-во_счетов']
X = safe_drop(X, ['кол-во_открытых_возобновляемых_счетов','кол-во_счетов_без_нарушений'])

if 'процент_счетов_без_просрочек' in X.columns:
    X['процент_счетов_без_просрочек'] = np.log1p(X['процент_счетов_без_просрочек'])
X = safe_drop(X, ['процент_счетов_без_просрочек','кол-во_возобновляемых_счетов_с_балансом_более_0',
                  'кол-во_карт_без_нарушений','кол-во_активных_возобновляемых_счетов',
                  'процент_счетов_прев_75_лимита'])

if 'пдн' in X.columns:
    X['пдн'] = X['пдн'].fillna(0)
X = safe_drop(X, ['кол-во_карт'])
if 'кол-во_месяцев_с_последнего_счета' in X.columns:
    X['кол-во_месяцев_с_последнего_счета'] = X['кол-во_месяцев_с_последнего_счета'].fillna(0)
if 'лимит_по_аннуитетным_счетам' in X.columns:
    X['лимит_по_аннуитетным_счетам'] = X['лимит_по_аннуитетным_счетам'].fillna(0)
    X['лимит_по_аннуитетным_счетам'] = np.log1p(X['лимит_по_аннуитетным_счетам'])
if 'индекс_проживания' in X.columns:
    X['индекс_проживания'] = X['индекс_проживания'].fillna(X['индекс_проживания'].median())
if 'Кол-во_без_нарушений/Кол-во_счетов' in X.columns:
    X['Кол-во_без_нарушений/Кол-во_счетов'] = X['Кол-во_без_нарушений/Кол-во_счетов'].fillna(0)
if 'кол-во_месяцев_с_последней_просрочки' in X.columns:
    X['кол-во_месяцев_с_последней_просрочки'] = X['кол-во_месяцев_с_последней_просрочки'].fillna(0)

from sklearn.preprocessing import MinMaxScaler
cols_to_sum = [
    'кол-во_возоб_счетов_за_2_года',
    'кол-во_возоб_счетов_за_год',
    'кол-во_возобновляемых_счетов_с_балансом_более_0',
    'кол-во_активных_возобновляемых_счетов',
    'кол-во_открытых_счетов_за_2_года',
    'кол-во_открытых_счетов_за_полгода',
    'кол-во_счетов_за_посл_год'
]
cols_to_sum_present = [c for c in cols_to_sum if c in test.columns]
if cols_to_sum_present:
    scaler=MinMaxScaler()
    _ = scaler.fit_transform(test[cols_to_sum_present])
    test['суммарное_кол-во_счетов'] = test[cols_to_sum_present].sum(axis=1)
X = pd.concat([
    X,
    test['суммарное_кол-во_счетов'] if 'суммарное_кол-во_счетов' in test.columns else pd.Series(0, index=X.index),
    test['кол-во_заявок_за_полгода'].fillna(0) if 'кол-во_заявок_за_полгода' in test.columns else pd.Series(0, index=X.index),
    test['кол-во_месяцев_с_первого_возобновляемого_счета'].fillna(0) if 'кол-во_месяцев_с_первого_возобновляемого_счета' in test.columns else pd.Series(0, index=X.index),
    test['процент_счетов_прев_75_лимита'].fillna(0) if 'процент_счетов_прев_75_лимита' in test.columns else pd.Series(0, index=X.index)
], axis=1)

if 'процентная_ставка' in X.columns:
    X['процентная_ставка'] = X['процентная_ставка'].fillna(X['процентная_ставка'].median())

categorical_cols = test.select_dtypes(include=['object']).columns.tolist()
X = pd.concat([X, test[categorical_cols]], axis=1)

missing = X.isnull().sum().sort_values()
missing

if 'срок_займа' in X.columns:
    X['срок_займа'] = X['срок_займа'].str.split().str[0].astype(int)

rating_order = ['А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж']
dop_rating_order = [f"{letter}{num}" for letter in rating_order for num in range(1, 6)]
enc_rating = OrdinalEncoder(categories=[rating_order, dop_rating_order])
if all(c in X.columns for c in ['рейтинг','допрейтинг']):
    X[['рейтинг', 'допрейтинг']] = enc_rating.fit_transform(X[['рейтинг','допрейтинг']])

if 'стаж' in X.columns:
    X['стаж'] = X['стаж'].fillna('не указан')
    experience_order = [['не указан','< 1 года', '1 год', '2 года', '3 года', '4 года', '5 лет', '6 лет', '7 лет', '8 лет', '9 лет', '10+ лет']]
    enc1 = OrdinalEncoder(categories=experience_order)
    X['стаж'] = enc1.fit_transform(X[['стаж']])

X = safe_drop(X, 'платежный_график')

if 'дата_первого_займа' in X.columns:
    X['дата_первого_займа'] = pd.to_datetime(X['дата_первого_займа'])
    X['год'] = X['дата_первого_займа'].dt.year
    X['декада'] = (X['год'] // 10) * 10

    enc_dek = OrdinalEncoder(categories=[[1930,1940,1950,1960,1970,1980,1990,2000,2010,2020]])
    X['декада'] = enc_dek.fit_transform(X[['декада']])
    X = safe_drop(X, 'дата_первого_займа')

if 'первоначальный_статус_займа' in X.columns:
    enc_init = OrdinalEncoder(categories=[['а','б']])
    X['первоначальный_статус_займа'] = enc_init.fit_transform(X[['первоначальный_статус_займа']])

X = safe_drop(X, 'пени_за_дефолт')

if 'совокупный_статус_подтверждения_доходов_заемщиков' in X.columns:
    X['совокупный_статус_подтверждения_доходов_заемщиков'] = X['совокупный_статус_подтверждения_доходов_заемщиков'].fillna('Не подтвержден')
    X['совокупный_статус_подтверждения_доходов_заемщиков'] = X['совокупный_статус_подтверждения_доходов_заемщиков'].astype(str)

X = safe_drop(X, 'особая_ситуация')

if 'процентная_ставка' in X.columns:
    X = X.dropna(subset=['процентная_ставка'])

if 'сумма_выплат_по_просрочкам' in X.columns:
    X['бинарные_идентификатор_суммы_просрочки'] = (X['сумма_выплат_по_просрочкам'] > 0).astype(int)
    X['сумма_выплат_по_просрочкам'] = np.log1p(X['сумма_выплат_по_просрочкам'].fillna(0))

if 'итоговый_статус_займа' in X.columns:
    y = X['итоговый_статус_займа']
    X = X.drop('итоговый_статус_займа', axis=1)


encoder_vl = OneHotEncoder(sparse_output=False, dtype=int, handle_unknown='ignore')
if 'владение_жильем' in X.columns:
    encoded_array = encoder_vl.fit_transform(X[['владение_жильем']])
    encoded_cols = encoder_vl.get_feature_names_out(['владение_жильем'])
    train_encoded = pd.DataFrame(encoded_array, columns=encoded_cols, index=X.index)
    X = pd.concat([X, train_encoded], axis=1)
    X = X.drop('владение_жильем', axis=1)

encoder_goal = OneHotEncoder(sparse_output=False, dtype=int, handle_unknown='ignore')
if 'цель_займа' in X.columns:
    encoded_array = encoder_goal.fit_transform(X[['цель_займа']])
    encoded_cols = encoder_goal.get_feature_names_out(['цель_займа'])
    train_encoded1 = pd.DataFrame(encoded_array, columns=encoded_cols, index=X.index)
    X = pd.concat([X, train_encoded1], axis=1)
    X = X.drop('цель_займа', axis=1)

encoder5 = OneHotEncoder(sparse_output=False, dtype=int, handle_unknown='ignore')
if 'подтвержден_ли_доход' in X.columns:
    encoded_array5 = encoder5.fit_transform(X[['подтвержден_ли_доход']])
    encoded_cols5 = encoder5.get_feature_names_out(['подтвержден_ли_доход'])
    train_encoded5 = pd.DataFrame(encoded_array5, columns=encoded_cols5, index=X.index)
    X = pd.concat([X, train_encoded5], axis=1)
    X = X.drop('подтвержден_ли_доход', axis=1)

encoder3 = OneHotEncoder(sparse_output=False, dtype=int, handle_unknown='ignore')
if 'совокупный_статус_подтверждения_доходов_заемщиков' in X.columns:
    encoded_array3 = encoder3.fit_transform(X[['совокупный_статус_подтверждения_доходов_заемщиков']])
    encoded_cols3 = encoder3.get_feature_names_out(['совокупный_статус_подтверждения_доходов_заемщиков'])
    train_encoded3 = pd.DataFrame(encoded_array3, columns=encoded_cols3, index=X.index)
    X = pd.concat([X, train_encoded3], axis=1)
    X = X.drop('совокупный_статус_подтверждения_доходов_заемщиков', axis=1)

encoder4 = OneHotEncoder(sparse_output=False, dtype=int, handle_unknown='ignore')
if 'тип_предоставления_кредита' in X.columns:
    encoded_array4 = encoder4.fit_transform(X[['тип_предоставления_кредита']])
    encoded_cols4 = encoder4.get_feature_names_out(['тип_предоставления_кредита'])
    train_encoded4 = pd.DataFrame(encoded_array4, columns=encoded_cols4, index=X.index)
    X = pd.concat([X, train_encoded4], axis=1)
    X = X.drop('тип_предоставления_кредита', axis=1)

encoder6 = OneHotEncoder(sparse_output=False, dtype=int, handle_unknown='ignore')
if 'пос_стоп_фактор' in X.columns:
    encoded_array6 = encoder6.fit_transform(X[['пос_стоп_фактор']])
    encoded_cols6 = encoder6.get_feature_names_out(['пос_стоп_фактор'])
    train_encoded6 = pd.DataFrame(encoded_array6, columns=encoded_cols6, index=X.index)
    X = pd.concat([X, train_encoded6], axis=1)
    X = X.drop('пос_стоп_фактор', axis=1)

encoder7 = OneHotEncoder(sparse_output=False, dtype=int, handle_unknown='ignore')
if 'юридический_статус' in X.columns:
    encoded_array7 = encoder7.fit_transform(X[['юридический_статус']])
    encoded_cols7 = encoder7.get_feature_names_out(['юридический_статус'])
    train_encoded7 = pd.DataFrame(encoded_array7, columns=encoded_cols7, index=X.index)
    X = pd.concat([X, train_encoded7], axis=1)
    X = X.drop('юридический_статус', axis=1)

encoder2 = OneHotEncoder(sparse_output=False, dtype=int, handle_unknown='ignore')
if 'тип_займа' in X.columns:
    encoded_array2 = encoder2.fit_transform(X[['тип_займа']])
    encoded_cols2 = encoder2.get_feature_names_out(['тип_займа'])
    train_encoded2 = pd.DataFrame(encoded_array2, columns=encoded_cols2, index=X.index)
    X = pd.concat([X, train_encoded2], axis=1)
    X = X.drop('тип_займа', axis=1)

new_cols = {}
if 'кол-во_ипотек' in test.columns:
    new_cols['log_ипотек'] = np.log1p(test['кол-во_ипотек'])
if 'кредитный_баланс_без_ипотеки' in test.columns:
    new_cols['log_кредит_без_ипотеки'] = np.log1p(test['кредитный_баланс_без_ипотеки'])
if 'аннуитет' in test.columns:
    new_cols['log_аннуитет'] = np.log1p(test['аннуитет'])
if all(c in X.columns for c in ['процентная_ставка','срок_займа','сумма_займа']):
    new_cols['процент_ставка_срок_сумма'] = X['процентная_ставка'] * X['срок_займа'] * X['сумма_займа']
if 'грейд_на_детерминаторе' in test.columns:
    new_cols['грейд_на_детерминаторе'] = test['грейд_на_детерминаторе']
if all(c in X.columns for c in ['годовой_доход','сумма_займа']):
    new_cols['доход_к_займу'] = X['годовой_доход'] / X['сумма_займа'].replace(0, np.nan)

if new_cols:
    X = pd.concat([X, pd.DataFrame(new_cols, index=X.index)], axis=1)

X.columns = X.columns.astype(str)
X['log_ипотек']=X['log_ипотек'].fillna(X['log_ипотек'].median())
X['log_кредит_без_ипотеки']=X['log_кредит_без_ипотеки'].fillna(X['log_кредит_без_ипотеки'].median())
X['кол-во_месяцев_с_последней_карты']=X['кол-во_месяцев_с_последней_карты'].fillna(X['кол-во_месяцев_с_последней_карты'].median())
X['кол-во_месяцев_с_последнего_возобновляемого_счета']=X['кол-во_месяцев_с_последнего_возобновляемого_счета'].fillna(X['кол-во_месяцев_с_последнего_возобновляемого_счета'].median())
X['кол-во_открытых_счетов_за_2_года']=X['кол-во_открытых_счетов_за_2_года'].fillna(X['кол-во_открытых_счетов_за_2_года'].median())
if 'регион' in X.columns:
    X['регион_target_encoded'] = X['регион'].map(region_means).fillna(global_mean)
    X['регион_freq_encoded'] = X['регион'].map(region_counts).fillna(0)
    X = X.drop('регион', axis=1)

if 'профессия_заемщика' in X.columns:
    X = X.drop('профессия_заемщика', axis=1)
    

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler

numeric_cols = X.select_dtypes(include=['int64','float64','int32']).columns
X = preprocessor.transform(X)

C:\Users\lego-\AppData\Local\Temp\ipykernel_19392\2025430610.py:131: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  X['дата_первого_займа'] = pd.to_datetime(X['дата_первого_займа'])


In [4]:
answer = pd.DataFrame()
test_predict = model.predict_proba(X)[:, 1]
answer['id'] = test['id']
answer['proba'] = test_predict
answer.to_csv('submission.csv', index=False)